# Defi IA 2023 - Prédiction du prix d'une chambre d'hôtels

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import urllib.parse
import requests
import os

In [2]:
domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)
user_id = '88be3640-82ae-46c6-ac59-92a9679d453d'

## Création d'un avatar

In [39]:
name = 'leS' #donner le nom de l'avatar
new_avatar = requests.post(path(f'avatars/{user_id}/{name}'))

In [40]:
avatar_info = requests.get(path(f"avatars/{user_id}"))
avatar_names = []

for avatar in avatar_info.json():
    avatar_names.append(avatar['name'])

avatar_names

['laulau',
 'Nini',
 'Tess',
 'john',
 'elisa',
 'alal',
 'timon',
 'alexandre',
 'audeetcharles',
 'perlou',
 'jojo',
 'jc',
 'pumba',
 'sylvie',
 'didier',
 'ratass',
 'shakira',
 'manon',
 'larouste',
 'coco',
 'nicolas',
 'marie',
 'bgdu31',
 'bulgare',
 'avatar_35',
 'avatar25',
 'allcity',
 'avatar40',
 'romanian_amsterdam',
 'noworlate',
 'moyen2',
 'madridvienna2',
 'europesoon',
 'sofia35',
 'avatar15',
 'avatar5',
 'avatarparisrome',
 'moyen',
 'now',
 'paris',
 'madridvienna',
 'romanianamsterdam',
 'belgian',
 'hongrois',
 'le_c',
 'austrian',
 'b',
 'led',
 'lec',
 'lel',
 'leS',
 'leL']

## Excécution des requêtes

In [42]:
avatar= "leS"
cities = ['amsterdam','copenhaguen','madrid','valletta']
languages = ['slovene']
mobiles = [0,1]
dates = [0,1,2,3,4,5,6,7]


if not os.path.exists('requests2'):
    os.mkdir('requests2')


#listes contenant toutes les combinaisons possibles pour un avatar
combination=[]
for count1,date in enumerate(dates):
    for count2,city in enumerate(cities):
        for count3,language in enumerate(languages):
            for count4,mobile in enumerate(mobiles):
                combination.append((city,date,language,mobile))
                params = {
                "avatar_name": avatar,
                "language": language,
                "city": city,
                "date": date,
                "mobile": mobile,}

                ## REQUÊTE ##
                requete = requests.get(path(f"pricing/{user_id}"), params=params)
                requete.json()
                pricing_requests = []

                request = [requete]
                for requete in request:
                    pricing_requests.append(
                        pd.DataFrame(requete.json()['prices']).assign(**requete.json()['request'])
                    )

                pricing_requests = pd.concat(pricing_requests)
                pricing_requests.to_csv(os.path.join('requests2', str(avatar) +'_'+str(city)+ '_' + str(date) + '_' + str(language) + '_' + str(mobile)+'.csv'))

KeyError: 'prices'

In [ ]:
requete.json()

## Automatisation du code pour création des datasets

On va chercher à créer une liste contenant le nom de tous les avatars.

In [16]:
r = requests.get(path(f"avatars/{user_id}"))
list_avatars = []
for avatar in r.json():
    list_avatars.append(avatar['name'])

Ayant tous les avatars, on peut créer les datasets pour chacun des avatars, s'il n'existe pas déjà, et les compléter avec les données récupérées.

In [17]:
hotels_features = pd.read_csv(os.path.join('features', 'features_hotels.csv'))

Le code ci-dessous permet de créer le dataset associé à un avatar précis. 

In [18]:
list_requests = glob.glob(os.path.join('requests2', '*.csv'))
list_requests = [request.split('\\')[-1] for request in list_requests]
df = pd.DataFrame([])

if not os.path.exists('datasets2'):
    os.mkdir('datasets2')

i= 0
for avatar in ["led"]: ############## Nom avatar à changer
    for request in list_requests:
        name_avatar_request = request.split('_')[0] 
        if (avatar==name_avatar_request):

            if not os.path.exists(os.path.join('datasets2', avatar + '_data.csv')):
                request_avatar = pd.read_csv(os.path.join('requests2', request))
                features_avatar = hotels_features.loc[hotels_features['hotel_id'].loc[request_avatar['hotel_id']]]
                features_avatar = features_avatar.reset_index()
                df = pd.concat([features_avatar, request_avatar[['price', 'stock', 'date', 'language', 'mobile']]], axis = 1)
                df['avatar_name'] = [avatar for i in range(len(df.index))]
                df = df.drop(columns = ['index'])
                df.to_csv(os.path.join('datasets2', avatar + '_data.csv'), index=False)
            else:
                df_old = pd.read_csv(os.path.join('datasets2', avatar + '_data.csv'))
                request_avatar = pd.read_csv(os.path.join('requests2', request))
                features_avatar = hotels_features.loc[hotels_features['hotel_id'].loc[request_avatar['hotel_id']]]
                features_avatar = features_avatar.reset_index()
                df_aux = pd.concat([features_avatar, request_avatar[['price', 'stock', 'date', 'language', 'mobile']]], axis = 1)
                df = pd.concat([df,df_aux], axis = 0)
                df = df.drop(columns = ['index'])

                df['avatar_name'] = [avatar for i in range(len(df.index))]
                df.to_csv(os.path.join('datasets2', avatar + '_data.csv'), index=False)
        
    df = pd.read_csv(os.path.join('datasets2', avatar + '_data.csv'))
    liste_date_unique = list(np.sort(np.unique(df['date'].values))[::-1])
    liste_date = np.sort(df['date'].values)[::-1]

    order_request_list = []
    for i in df.index:
        date = df.loc[i]['date']
        order_request = liste_date_unique.index(date) + 1
        order_request_list.append(int(order_request))

    df['order_requests'] = order_request_list
    df.to_csv(os.path.join('datasets2', avatar + '_data.csv'), index=False)

In [19]:
datasets = ['datasets2/led_data.csv'] ############## Nom avatar à changer : NOMAVATAR_data.csv
for dataset_path in datasets:
    dataset = pd.read_csv(dataset_path)
    if not os.path.exists('data2.csv'):
        df = pd.DataFrame([])
        df = dataset
        df.to_csv('data2.csv', index= False)
    else:
        df = pd.read_csv('data2.csv')
        df = pd.concat([df, dataset], axis = 0)
        df.to_csv('data2.csv', index=False)


In [20]:
df1 = pd.read_csv('data.csv')
df2 = pd.read_csv('data2.csv')

df3 = pd.concat([df1,df2], axis = 0)
df3.to_csv('data3.csv', index=False)

# Récupération des anciennes données et ajout d'order requests


In [141]:
data_nini = pd.read_csv('datasets/Tess_data.csv')
data_nini = data_nini.sort_values(by = ['date', 'language'], ascending = False)
data_nini.head()

,hotel_id,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile,avatar_name,order_requests
0,519,Yin Yang,Royal Lotus,copenhagen,0,0,0,208,68,30,french,0,laulau,1
1,523,Independant,Independant,copenhagen,0,0,0,85,41,30,french,0,laulau,1
2,898,Yin Yang,8 Premium,copenhagen,1,0,0,86,24,30,french,0,laulau,1
3,770,Morriott International,Morriot,copenhagen,1,1,0,210,73,30,french,0,laulau,1
4,562,Chillton Worldwide,Quadrupletree,copenhagen,0,0,0,215,74,30,french,0,laulau,1


In [131]:
len(data_nini.index)

98716

In [132]:
dates = data_nini['date']
languages = data_nini['language']

list_order_requests = [1]

order_request = 1

for i in range(1,len(data_nini.index)):
    if data_nini.iloc[i-1]['date'] == data_nini.iloc[i]['date'] and data_nini.iloc[i-1]['language'] == data_nini.iloc[i]['language']:
        list_order_requests.append(order_request)
    else:
        order_request += 1
        list_order_requests.append(order_request)
print(len(list_order_requests))

98716


In [133]:
data_nini['order_requests'] = list_order_requests
data_nini.head()

,hotel_id,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile,avatar_name,order_requests
0,519,Yin Yang,Royal Lotus,copenhagen,0,0,0,208,68,30,french,0,laulau,1
1,523,Independant,Independant,copenhagen,0,0,0,85,41,30,french,0,laulau,1
2,898,Yin Yang,8 Premium,copenhagen,1,0,0,86,24,30,french,0,laulau,1
3,770,Morriott International,Morriot,copenhagen,1,1,0,210,73,30,french,0,laulau,1
4,562,Chillton Worldwide,Quadrupletree,copenhagen,0,0,0,215,74,30,french,0,laulau,1


In [134]:
data_nini.to_csv('datasets/Tess_data.csv', index=False)

In [143]:

df2.head()

,hotel_id,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile,avatar_name,order_requests
0,515,Accar Hotels,Safitel,amsterdam,1,1,0,375,22,16,finnish,1,alal,7
1,902,Yin Yang,Royal Lotus,amsterdam,1,1,0,220,23,16,finnish,1,alal,7
2,867,Independant,Independant,amsterdam,1,0,0,167,12,16,finnish,1,alal,7
3,303,Yin Yang,Royal Lotus,amsterdam,1,1,0,240,19,16,finnish,1,alal,7
4,234,Boss Western,J.Halliday Inn,amsterdam,0,0,0,197,14,16,finnish,1,alal,7


In [144]:
df1.head()

,hotel_id,group,brand,city,parking,pool,children_policy,price,stock,date,language,mobile,avatar_name,order_requests
0,519,Yin Yang,Royal Lotus,copenhagen,0,0,0,208,68,30,french,0,laulau,1
1,523,Independant,Independant,copenhagen,0,0,0,85,41,30,french,0,laulau,1
2,898,Yin Yang,8 Premium,copenhagen,1,0,0,86,24,30,french,0,laulau,1
3,770,Morriott International,Morriot,copenhagen,1,1,0,210,73,30,french,0,laulau,1
4,562,Chillton Worldwide,Quadrupletree,copenhagen,0,0,0,215,74,30,french,0,laulau,1
